In [2]:
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_curve

C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [7]:
# -*- coding: utf-8 -*-
"""
Year-wise UH simulation + ML classification (Excel input, already hourly)
POINT METRICS ONLY (no event-day logic)
- Expects columns: "Date Time", "WL (mMSL)", "precip", "Flash Flood" (0/1)
- Works year-by-year (UH reset each Jan 1)
- Chronological split (80/20) within each year
- Saves yearly point metrics, PR curves, hydrographs, and summary Excel
"""

# ========================
# CONFIG (edit paths if needed)
# ========================
INPUT_XLSX          = "PREMONSOON-ALL-FILLEDv4.xlsx"   # <-- put your Excel filename here
SHEET               ="mar-may"
OUTPUT_XLSX         = "calibration_yearwise_results.xlsx"
TIMESTAMP_COL       = "Date Time"
RAIN_COL            = "precip"
LABEL_COL           = "Flash Flood"
# Identify available observed series name (for overlay)
OBS_COL = "WL (mMSL)"

    # time base (hours)
UH_A_candidates     = [0.01,0.05,0.1,0.2,0.3,0.4]       # alpha (rising fraction of the triangle)
                       # skip very short years

# ========================
# Imports
# ========================
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_curve

# ========================
# Helper functions
# ========================
def triangular_uh(width_h:int, alpha:float=0.5):
    """Triangular UH kernel with separate rising fraction alpha."""
    width_h = int(max(2, width_h))
    alpha = float(np.clip(alpha, 0.01, 0.99))
    r_len = max(1, int(np.ceil(width_h * alpha)))
    f_len = max(1, width_h - r_len)
    up = np.linspace(1, r_len, r_len, dtype=float)
    down = np.linspace(f_len, 1, f_len, dtype=float)
    k = np.concatenate([up, down[1:]]) if f_len > 0 else up
    return k / k.sum()

def uh_conv_trailing(r: pd.Series, width_h:int, alpha:float=0.5) -> pd.Series:
    """Trailing (causal) UH convolution; output index aligns with input."""
    k = triangular_uh(width_h, alpha)
    v = np.convolve(r.fillna(0.0).values, k, mode="full")[:len(r)]
    return pd.Series(v, index=r.index)

def compute_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall    = tp / (tp + fn) if (tp + fn) else 0.0
    f1        = 2*precision*recall/(precision+recall) if (precision+recall) else 0.0
    far       = fp / (fp + tn) if (fp + tn) else 0.0
    return precision, recall, f1, far

def load_excel_already_hourly(path, ts_col, rain_col, label_col):
    """Load your already-hourly dataset and ensure types are correct."""
    df = pd.read_excel(path,sheet_name=SHEET)
    df[ts_col] = pd.to_datetime(df[ts_col])
    df = df.sort_values(ts_col).drop_duplicates(subset=[ts_col])

    # Ensure numeric for used cols
    df[rain_col]  = pd.to_numeric(df[rain_col], errors="coerce").fillna(0.0)
    df[label_col] = pd.to_numeric(df[label_col], errors="coerce").fillna(0).astype(int)
    return df
def peak_lag_hours(sim: pd.Series, obs: pd.Series) -> float:
    """Time difference (hours) between the max of sim and max of obs.
    >0 ⇒ sim peaks later; <0 ⇒ sim peaks earlier."""
    s = pd.to_numeric(sim, errors="coerce")
    o = pd.to_numeric(obs, errors="coerce")
    both = pd.concat([s, o], axis=1).dropna()
    if both.empty:
        return np.nan
    sim_peak_t = both.index[both.iloc[:,0].argmax()]
    obs_peak_t = both.index[both.iloc[:,1].argmax()]
    return (sim_peak_t - obs_peak_t).total_seconds() / 3600.0

# ========================
# Load data
# ========================
df = load_excel_already_hourly(INPUT_XLSX,TIMESTAMP_COL, RAIN_COL, LABEL_COL)

# Make folders
os.makedirs("models_marmay", exist_ok=True)
os.makedirs("plots_marmay", exist_ok=True)
os.makedirs("hydrographs_marmay", exist_ok=True)


# ========================
# Year-wise grid search
# ========================
years = sorted(df[TIMESTAMP_COL].dt.year.unique())

results_yearly = []        # per (year, W, A, model)
summary_all = []           # per (W, A, model) across years


for A in UH_A_candidates:
    W=6/A
    

    for yr in years:
        
        full_path=os.path.join("hydrographs_marmay",str(yr))
        os.makedirs(full_path, exist_ok=True)
        df_y = df[df[TIMESTAMP_COL].dt.year == yr].copy()
        if df_y.empty:
            continue

        df_y = df_y.sort_values(TIMESTAMP_COL).set_index(TIMESTAMP_COL)

        # --- UH reset within the year ---
        df_y["x"] = uh_conv_trailing(df_y[RAIN_COL].reset_index(drop=True), width_h=W, alpha=A).values

       
       

        # ---------- plot: hydrograph per year ----------
        try:
            
            fig, ax1 = plt.subplots(figsize=(12, 4))
            ax1.plot(df_y.index, df_y["x"], linewidth=1.2, label=f"Sim UH (W={W}, A={A})")
            ax1.set_xlabel("Time")
            ax1.set_ylabel("Simulated hydrograph (mm/hr units)")
            ax1.grid(True, linestyle="--", alpha=0.5)

       
        

            # water level on secondary axis
            ax2 = ax1.twinx()
            ax2.plot(df_y.index, df_y[OBS_COL].values, linewidth=1, color='grey',linestyle='--',label=f"Observed {OBS_COL}")
            ax2.set_ylabel(f"{OBS_COL}")
            
            # --- correlation (units are different → standardize first) ---
            def _z(s):
                s = pd.to_numeric(s, errors="coerce")
                s = s.reindex(df_y.index)       # align just in case
                s = s.dropna()
                return (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) != 0 else 1)
            
            # Align indices, drop NaNs
            pair = pd.concat([df_y["x"], df_y[OBS_COL]], axis=1).dropna()
            sim_z = _z(pair["x"])
            obs_z = _z(pair[OBS_COL])
            
            # Plain (same-time) Pearson correlation on z-scores
            r = sim_z.corr(obs_z)
            # Peak-time difference (hours) for this (W, A, year)
            peak_diff_h = peak_lag_hours(df_y["x"], df_y[OBS_COL])

            # Save correlation values for summary
            results_yearly.append({
                "Year": yr,
                "UH_W": W,
                "UH_A": A,
                "r_same_time": r,
                "peak_time_diff_h": peak_diff_h   # <-- NEW
            })
            

           # Annotate on the plot (moved to top-right so it doesn't clash with legends)
            txt = f"r (z-score) = {r:.2f}"
            ax1.text(
                0.99, 0.98, txt,
                transform=ax1.transAxes, va="top", ha="right",
                fontsize=9, bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7, lw=0.5)
            )
            
            # Build two *separate* legends and place them vertically without overlap
            # Top-left: ax1 series
            lines1, labels1 = ax1.get_legend_handles_labels()
            if labels1:
                leg1 = ax1.legend(lines1, labels1, loc="upper left", frameon=True)
                ax1.add_artist(leg1)  # ensure it stays when we add the second legend
            
            # Bottom-left: ax2 series
            lines2, labels2 = ax2.get_legend_handles_labels()
            if labels2:
                ax2.legend(lines2, labels2, loc="lower left", frameon=True)
            
            plt.title(f"Hydrograph | Year={yr} | W={W}, A={A}")
            out_png = os.path.join(full_path, f"Hydrograph_Year{yr}_UH_W{W}_UH_A{A}.png")
            plt.savefig(out_png, dpi=300, bbox_inches="tight")
            plt.close()

        except Exception as e:
            print(f"[Plot hydrograph] Year {yr}, W={W}, A={A} failed: {e}")

            


In [ ]:
# -*- coding: utf-8 -*-
"""
UH simulation + SEASONAL baseflow calibration
- One fixed beta for March, April, May (same across all years)
- Beta ∈ [0.5, 2.0]
- Calibration maximizes correlation with water level (all years pooled)
"""

# ========================
# CONFIG
# ========================
INPUT_XLSX    = "PREMONSOON-ALL-FILLEDv4.xlsx"
SHEET         = "may-july"
TIMESTAMP_COL = "Date Time"
RAIN_COL      = "precip"
OBS_COL       = "WL (mMSL)"

UH_W     = 120
UH_ALPHA = 0.05

BASEFLOW_WINDOW_DAYS = 30
BETA_VALUES = np.arange(0.5, 2.01, 0.05)

MONTHS = {5: "May", 6: "June", 7: "July"}

# ========================
# Imports
# ========================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ========================
# Helper functions
# ========================
def triangular_uh(width_h, alpha):
    width_h = int(max(2, width_h))
    alpha = float(np.clip(alpha, 0.01, 0.99))
    r_len = max(1, int(np.ceil(width_h * alpha)))
    f_len = max(1, width_h - r_len)
    up = np.linspace(1, r_len, r_len)
    down = np.linspace(f_len, 1, f_len)
    k = np.concatenate([up, down[1:]])
    return k / k.sum()

def uh_conv_trailing(r, width_h, alpha):
    k = triangular_uh(width_h, alpha)
    v = np.convolve(r.fillna(0).values, k, mode="full")[:len(r)]
    return pd.Series(v, index=r.index)

def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    return (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) else 1)

# ========================
# Load data
# ========================
df = pd.read_excel(INPUT_XLSX, sheet_name=SHEET)
df[TIMESTAMP_COL] = pd.to_datetime(df[TIMESTAMP_COL])
df = df.sort_values(TIMESTAMP_COL)

df[RAIN_COL] = pd.to_numeric(df[RAIN_COL], errors="coerce").fillna(0.0)
df[OBS_COL]  = pd.to_numeric(df[OBS_COL], errors="coerce")

df["year"]  = df[TIMESTAMP_COL].dt.year
df["month"] = df[TIMESTAMP_COL].dt.month
df = df.set_index(TIMESTAMP_COL)

# ========================
# Compute UH and baseflow proxy (ONCE)
# ========================
df["x"] = uh_conv_trailing(df[RAIN_COL], UH_W, UH_ALPHA)

df["x_base"] = (
    df["x"]
    .rolling(window=BASEFLOW_WINDOW_DAYS * 24, min_periods=1)
    .mean()
)

# ========================
# CALIBRATE ONE BETA PER MONTH (ALL YEARS)
# ========================
beta_month = {}
summary = []

for m, mname in MONTHS.items():

    df_m = df[df["month"] == m]

    pair = pd.concat(
        [df_m["x"], df_m["x_base"], df_m[OBS_COL]],
        axis=1
    ).dropna()

    best_r = -np.inf
    best_beta = np.nan

    for beta in BETA_VALUES:
        x_tot = pair["x"] + beta * pair["x_base"]
        r_try = zscore(x_tot).corr(zscore(pair[OBS_COL]))

        if r_try > best_r:
            best_r = r_try
            best_beta = beta

    beta_month[m] = best_beta

    summary.append({
        "Month": mname,
        "Best_beta": best_beta,
        "Corr": best_r
    })

# ========================
# APPLY FIXED MONTHLY BETAS
# ========================
df["baseflow"] = 0.0
df["x_total"] = df["x"].copy()

for m, beta in beta_month.items():
    idx = df["month"] == m
    # baseflow column holds the additive component
    df.loc[idx, "baseflow"] = beta * df.loc[idx, "x_base"]
    df.loc[idx, "x_total"] = df.loc[idx, "x"] + df.loc[idx, "baseflow"]
print("df",df)
# ========================
# SAVE SUMMARY
# ========================
df_summary = pd.DataFrame(summary)
df_summary.to_excel("seasonal_baseflow_beta_summary.xlsx", index=False)

print("\nSeasonal baseflow factors (fixed across years):")
print(df_summary)

# ========================
# OPTIONAL: PLOT PER YEAR
# ========================
os.makedirs("hydrographs_marmay", exist_ok=True)
df_y1=pd.DataFrame()
for yr in sorted(df["year"].unique()):
    df_y = df[df["year"] == yr]
    print("df_y", df_y)
    # --- compute r for March–May only ---
    df_pm = df_y[df_y["month"].isin([3, 4, 5])]
    
    pair = pd.concat(
        [df_pm["x_total"], df_pm[OBS_COL]],
        axis=1
    ).dropna()
    
    def z(s):
        return (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) else 1)
    
    r_pm = z(pair["x_total"]).corr(z(pair[OBS_COL]))
    
    df_y1 = pd.concat([df_y1, df_y], axis=0)


    fig, ax1 = plt.subplots(figsize=(12, 4))

    ax1.plot(df_y.index, df_y["x_total"],
             lw=1.2, label="Simulated x + seasonal baseflow")
    ax1.set_ylabel("Simulated flow (proxy)")
    ax1.grid(True, alpha=0.4)
    
    ax2 = ax1.twinx()
    ax2.plot(df_y.index, df_y[OBS_COL],
             "--", color="grey", lw=1, label=OBS_COL)
    ax2.set_ylabel("Water level")
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")
    
    # ---- NOW add annotation ----
    txt = f"Mar–May r = {r_pm:.2f}"
    ax1.text(
        0.98, 0.90, txt,   # slightly inward
        transform=ax1.transAxes,
        ha="right", va="top",
        fontsize=9,
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
        zorder=10
    )
    
    plt.title(f"Hydrograph with Seasonal Baseflow | Year {yr}")
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # <-- leave space at top
    plt.savefig(
        f"hydrographs_mayjun/Hydrograph_{yr}_seasonal_baseflow.png",
        dpi=300,
        bbox_inches="tight"
    )
    plt.close()

df                      WL (mMSL)    precip  month  year  Flash Flood         x  \
Date Time                                                                      
2010-05-16 00:00:00   9.876000  0.600239      5  2010            0  0.000093   
2010-05-16 01:00:00   9.863500  0.063596      5  2010            0  0.000196   
2010-05-16 02:00:00   9.851000  0.036900      5  2010            0  0.000304   
2010-05-16 03:00:00   9.838500  0.023337      5  2010            0  0.000416   
2010-05-16 04:00:00   9.826000  0.128843      5  2010            0  0.000548   
...                        ...       ...    ...   ...          ...       ...   
2021-07-31 19:00:00   7.719167  0.466129      7  2021            0  0.757862   
2021-07-31 20:00:00   7.728333  0.417797      7  2021            0  0.754226   
2021-07-31 21:00:00   7.737500  0.645099      7  2021            0  0.745650   
2021-07-31 22:00:00   7.746667  0.956017      7  2021            0  0.738276   
2021-07-31 23:00:00   7.755833  1.356

In [4]:
df.to_excel('simulated_x_monsoon120.xlsx')

In [ ]:
MONSOON

In [3]:
# -*- coding: utf-8 -*-
"""
Year-wise UH simulation + ML classification (Excel input, already hourly)
POINT METRICS ONLY (no event-day logic)
- Expects columns: "Date Time", "WL (mMSL)", "precip", "Flash Flood" (0/1)
- Works year-by-year (UH reset each Jan 1)
- Chronological split (80/20) within each year
- Saves yearly point metrics, PR curves, hydrographs, and summary Excel
"""

# ========================
# CONFIG (edit paths if needed)
# ========================
INPUT_XLSX          = "PREMONSOON-ALL-FILLEDv4.xlsx"   # <-- put your Excel filename here
SHEET               ="may-july"
OUTPUT_XLSX         = "calibration_yearwise_MONSOONresults.xlsx"
TIMESTAMP_COL       = "Date Time"
RAIN_COL            = "precip"
LABEL_COL           = "Flash Flood"
# Identify available observed series name (for overlay)
OBS_COL = "WL (mMSL)"

    # time base (hours)
UH_A_candidates     = [0.01,0.02,0.03,0.04,0.05]       # alpha (rising fraction of the triangle)
                       # skip very short years

# ========================
# Imports
# ========================
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_curve

# ========================
# Helper functions
# ========================
def triangular_uh(width_h:int, alpha:float=0.5):
    """Triangular UH kernel with separate rising fraction alpha."""
    width_h = int(max(2, width_h))
    alpha = float(np.clip(alpha, 0.01, 0.99))
    r_len = max(1, int(np.ceil(width_h * alpha)))
    f_len = max(1, width_h - r_len)
    up = np.linspace(1, r_len, r_len, dtype=float)
    down = np.linspace(f_len, 1, f_len, dtype=float)
    k = np.concatenate([up, down[1:]]) if f_len > 0 else up
    return k / k.sum()

def uh_conv_trailing(r: pd.Series, width_h:int, alpha:float=0.5) -> pd.Series:
    """Trailing (causal) UH convolution; output index aligns with input."""
    k = triangular_uh(width_h, alpha)
    v = np.convolve(r.fillna(0.0).values, k, mode="full")[:len(r)]
    return pd.Series(v, index=r.index)

def compute_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall    = tp / (tp + fn) if (tp + fn) else 0.0
    f1        = 2*precision*recall/(precision+recall) if (precision+recall) else 0.0
    far       = fp / (fp + tn) if (fp + tn) else 0.0
    return precision, recall, f1, far

def load_excel_already_hourly(path, ts_col, rain_col, label_col):
    """Load your already-hourly dataset and ensure types are correct."""
    df = pd.read_excel(path,sheet_name=SHEET)
    df[ts_col] = pd.to_datetime(df[ts_col])
    df = df.sort_values(ts_col).drop_duplicates(subset=[ts_col])

    # Ensure numeric for used cols
    df[rain_col]  = pd.to_numeric(df[rain_col], errors="coerce").fillna(0.0)
    df[label_col] = pd.to_numeric(df[label_col], errors="coerce").fillna(0).astype(int)
    return df
def peak_lag_hours(sim: pd.Series, obs: pd.Series) -> float:
    """Time difference (hours) between the max of sim and max of obs.
    >0 ⇒ sim peaks later; <0 ⇒ sim peaks earlier."""
    s = pd.to_numeric(sim, errors="coerce")
    o = pd.to_numeric(obs, errors="coerce")
    both = pd.concat([s, o], axis=1).dropna()
    if both.empty:
        return np.nan
    sim_peak_t = both.index[both.iloc[:,0].argmax()]
    obs_peak_t = both.index[both.iloc[:,1].argmax()]
    return (sim_peak_t - obs_peak_t).total_seconds() / 3600.0

# ========================
# Load data
# ========================
df = load_excel_already_hourly(INPUT_XLSX,TIMESTAMP_COL, RAIN_COL, LABEL_COL)

# Make folders
os.makedirs("models_marmay", exist_ok=True)
os.makedirs("plots_mayjun", exist_ok=True)
os.makedirs("hydrographs_mayjun", exist_ok=True)


# ========================
# Year-wise grid search
# ========================
years = sorted(df[TIMESTAMP_COL].dt.year.unique())

results_yearly = []        # per (year, W, A, model)
summary_all = []           # per (W, A, model) across years


for A in UH_A_candidates:
    W=6/A
    

    for yr in years:
        
        full_path=os.path.join("hydrographs_mayjun",str(yr))
        os.makedirs(full_path, exist_ok=True)
        df_y = df[df[TIMESTAMP_COL].dt.year == yr].copy()
        if df_y.empty:
            continue

        df_y = df_y.sort_values(TIMESTAMP_COL).set_index(TIMESTAMP_COL)

        # --- UH reset within the year ---
        df_y["x"] = uh_conv_trailing(df_y[RAIN_COL].reset_index(drop=True), width_h=W, alpha=A).values

       
       

        # ---------- plot: hydrograph per year ----------
        try:
            
            fig, ax1 = plt.subplots(figsize=(12, 4))
            ax1.plot(df_y.index, df_y["x"], linewidth=1.2, label=f"Sim UH (W={W}, A={A})")
            ax1.set_xlabel("Time")
            ax1.set_ylabel("Simulated discharge (x) (mm/hr units)")
            ax1.grid(True, linestyle="--", alpha=0.5)

       
        

            # water level on secondary axis
            ax2 = ax1.twinx()
            ax2.plot(df_y.index, df_y[OBS_COL].values, linewidth=1, color='grey',linestyle='--',label=f"Observed {OBS_COL}")
            ax2.set_ylabel(f"{OBS_COL}")
            
            # --- correlation (units are different → standardize first) ---
            def _z(s):
                s = pd.to_numeric(s, errors="coerce")
                s = s.reindex(df_y.index)       # align just in case
                s = s.dropna()
                return (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) != 0 else 1)
            
            # Align indices, drop NaNs
            pair = pd.concat([df_y["x"], df_y[OBS_COL]], axis=1).dropna()
            sim_z = _z(pair["x"])
            obs_z = _z(pair[OBS_COL])
            
            # Plain (same-time) Pearson correlation on z-scores
            r = sim_z.corr(obs_z)
            # Peak-time difference (hours) for this (W, A, year)
            peak_diff_h = peak_lag_hours(df_y["x"], df_y[OBS_COL])

            # Save correlation values for summary
            results_yearly.append({
                "Year": yr,
                "UH_W": W,
                "UH_A": A,
                "r_same_time": r,
                "peak_time_diff_h": peak_diff_h   # <-- NEW
            })
            

           # Annotate on the plot (moved to top-right so it doesn't clash with legends)
            txt = f"r (z-score) = {r:.2f}"
            ax1.text(
                0.99, 0.98, txt,
                transform=ax1.transAxes, va="top", ha="right",
                fontsize=9, bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7, lw=0.5)
            )
            
            # Build two *separate* legends and place them vertically without overlap
            # Top-left: ax1 series
            lines1, labels1 = ax1.get_legend_handles_labels()
            if labels1:
                leg1 = ax1.legend(lines1, labels1, loc="upper left", frameon=True)
                ax1.add_artist(leg1)  # ensure it stays when we add the second legend
            
            # Bottom-left: ax2 series
            lines2, labels2 = ax2.get_legend_handles_labels()
            if labels2:
                ax2.legend(lines2, labels2, loc="lower left", frameon=True)
            
            plt.title(f"Hydrograph | Year={yr} | W={W}, A={A}")
            out_png = os.path.join(full_path, f"Hydrograph_Year_MONSOON-{yr}_UH_W{W}_UH_A{A}.png")
            plt.savefig(out_png, dpi=300, bbox_inches="tight")
            plt.close()

        except Exception as e:
            print(f"[Plot hydrograph] Year {yr}, W={W}, A={A} failed: {e}")

            


In [ ]:
# -*- coding: utf-8 -*-
"""
UH simulation + SEASONAL baseflow calibration
- One fixed beta for March, April, May (same across all years)
- Beta ∈ [0.5, 2.0]
- Calibration maximizes correlation with water level (all years pooled)
"""

# ========================
# CONFIG
# ========================
INPUT_XLSX    = "PREMONSOON-ALL-FILLEDv4.xlsx"
SHEET         = "may-july"
TIMESTAMP_COL = "Date Time"
RAIN_COL      = "precip"
OBS_COL       = "WL (mMSL)"

UH_W     = 600
UH_ALPHA = 0.01

BASEFLOW_WINDOW_DAYS = 30
BETA_VALUES = np.arange(0.5, 2.01, 0.05)

MONTHS = {5: "May",6:"June",7:"July"}

# ========================
# Imports
# ========================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ========================
# Helper functions
# ========================
def triangular_uh(width_h, alpha):
    width_h = int(max(2, width_h))
    alpha = float(np.clip(alpha, 0.01, 0.99))
    r_len = max(1, int(np.ceil(width_h * alpha)))
    f_len = max(1, width_h - r_len)
    up = np.linspace(1, r_len, r_len)
    down = np.linspace(f_len, 1, f_len)
    k = np.concatenate([up, down[1:]])
    return k / k.sum()

def uh_conv_trailing(r, width_h, alpha):
    k = triangular_uh(width_h, alpha)
    v = np.convolve(r.fillna(0).values, k, mode="full")[:len(r)]
    return pd.Series(v, index=r.index)

def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    return (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) else 1)

# ========================
# Load data
# ========================
df = pd.read_excel(INPUT_XLSX, sheet_name=SHEET)
df[TIMESTAMP_COL] = pd.to_datetime(df[TIMESTAMP_COL])
df = df.sort_values(TIMESTAMP_COL)

df[RAIN_COL] = pd.to_numeric(df[RAIN_COL], errors="coerce").fillna(0.0)
df[OBS_COL]  = pd.to_numeric(df[OBS_COL], errors="coerce")

df["year"]  = df[TIMESTAMP_COL].dt.year
df["month"] = df[TIMESTAMP_COL].dt.month
df = df.set_index(TIMESTAMP_COL)

# ========================
# Compute UH and baseflow proxy (ONCE)
# ========================
df["x"] = uh_conv_trailing(df[RAIN_COL], UH_W, UH_ALPHA)

df["x_base"] = (
    df["x"]
    .rolling(window=BASEFLOW_WINDOW_DAYS * 24, min_periods=1)
    .mean()
 )

# ========================
# CALIBRATE ONE BETA PER MONTH (ALL YEARS)
# ========================
beta_month = {}
summary = []

for m, mname in MONTHS.items():

    df_m = df[df["month"] == m]

    pair = pd.concat(
        [df_m["x"], df_m["x_base"], df_m[OBS_COL]],
        axis=1
    ).dropna()

    best_r = -np.inf
    best_beta = np.nan

    for beta in BETA_VALUES:
        x_tot = pair["x"] + beta * pair["x_base"]
        r_try = zscore(x_tot).corr(zscore(pair[OBS_COL]))

        if r_try > best_r:
            best_r = r_try
            best_beta = beta

    beta_month[m] = best_beta

    summary.append({
        "Month": mname,
        "Best_beta": best_beta,
        "Corr": best_r
    })

# ========================
# APPLY FIXED MONTHLY BETAS
# ========================
df["baseflow"] = 0.0
df["x_total"] = df["x"].copy()

for m, beta in beta_month.items():
    idx = df["month"] == m
    df.loc[idx, "baseflow"] = beta * df.loc[idx, "x_base"]
    df.loc[idx, "x_total"] = df.loc[idx, "x"] + df.loc[idx, "baseflow"]
print("df",df)
# ========================
# SAVE SUMMARY
# ========================
df_summary = pd.DataFrame(summary)
df_summary.to_excel("seasonal_baseflow_beta_summary.xlsx", index=False)

print("\nSeasonal baseflow factors (fixed across years):")
print(df_summary)

# ========================
# OPTIONAL: PLOT PER YEAR
# ========================
os.makedirs("hydrographs_mayjun", exist_ok=True)
df_y1=pd.DataFrame()
for yr in sorted(df["year"].unique()):
    df_y = df[df["year"] == yr]
    print("df_y", df_y)
    # --- compute r for March–May only ---
    df_pm = df_y[df_y["month"].isin([3, 4, 5])]
    
    pair = pd.concat(
        [df_pm["x_total"], df_pm[OBS_COL]],
        axis=1
    ).dropna()
    
    def z(s):
        return (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) else 1)
    
    r_pm = z(pair["x_total"]).corr(z(pair[OBS_COL]))
    
    df_y1 = pd.concat([df_y1, df_y], axis=0)


    fig, ax1 = plt.subplots(figsize=(12, 4))

    ax1.plot(df_y.index, df_y["x_total"],
             lw=1.2, label="Simulated x + seasonal baseflow")
    ax1.set_ylabel("Simulated flow (proxy)")
    ax1.grid(True, alpha=0.4)
    
    ax2 = ax1.twinx()
    ax2.plot(df_y.index, df_y[OBS_COL],
             "--", color="grey", lw=1, label=OBS_COL)
    ax2.set_ylabel("Water level")
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")
    
    # ---- NOW add annotation ----
    txt = f"Mar–May r = {r_pm:.2f}"
    ax1.text(
        0.98, 0.90, txt,   # slightly inward
        transform=ax1.transAxes,
        ha="right", va="top",
        fontsize=9,
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
        zorder=10
    )
    
    plt.title(f"Hydrograph with Seasonal Baseflow | Year {yr}")
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # <-- leave space at top
    plt.savefig(
        f"hydrographs_mayjun/Hydrograph_{yr}_seasonal_baseflow.png",
        dpi=300,
        bbox_inches="tight"
    )
    plt.close()

df                      WL (mMSL)    precip  month  year  Flash Flood         x  \
Date Time                                                                      
2010-05-16 00:00:00   9.876000  0.600239      5  2010            0  0.000003   
2010-05-16 01:00:00   9.863500  0.063596      5  2010            0  0.000007   
2010-05-16 02:00:00   9.851000  0.036900      5  2010            0  0.000011   
2010-05-16 03:00:00   9.838500  0.023337      5  2010            0  0.000015   
2010-05-16 04:00:00   9.826000  0.128843      5  2010            0  0.000020   
...                        ...       ...    ...   ...          ...       ...   
2021-07-31 19:00:00   7.719167  0.466129      7  2021            0  0.631775   
2021-07-31 20:00:00   7.728333  0.417797      7  2021            0  0.631310   
2021-07-31 21:00:00   7.737500  0.645099      7  2021            0  0.629847   
2021-07-31 22:00:00   7.746667  0.956017      7  2021            0  0.628605   
2021-07-31 23:00:00   7.755833  1.356

In [10]:
df.to_excel('simulated_x_monsoon600.xlsx')